### File structure under PeopleSnapShot


* `consensus.pkl` → the main file containing detected **SMPL pose, shape, and camera parameters** for the subject.
  * betas – the shape coefficients (10-D) describing the subject’s static body shape. Identical to standard SMPL shape parameters.
  * v_personal – a per-vertex offset array of shape (6890, 3).This stores small corrective displacements from the neutral SMPL template to the person’s fitted mesh (e.g., clothing bulges, hair, etc.).
  * When you reconstruct the canonical mesh:
  
    ```
    vertices = smpl_model(betas=betas).vertices[0] + v_personal
    ```

    you get the personalized “consensus” geometry that best fits all frames for that subject.
* `reconstructed_poses.hdf5` — stores per-frame SMPL parameters for the full sequence (same structure but over time).
  | Key         | Shape     | Meaning                                                                 |
  | ----------- | --------- | ----------------------------------------------------------------------- |
  | **`betas`** | (10,)     | Static body-shape coefficients, same for all frames.                    |
  | **`pose`**  | (649, 72) | SMPL pose parameters for each of 649 frames (24 joints × 3 axis-angle). |
  | **`trans`** | (649, 3)  | Global translation of the body root per frame.                          |

* `camera.pkl` — intrinsic/extrinsic camera parameters for projection.
* `consensus.obj` — the reconstructed mesh generated from the SMPL params.
* `tex-female-3-casual.jpg` — UV texture corresponding to the mesh.

In [5]:
pwd

'/Users/lemon/Documents/TUD/Thesis/Code/avatar-benchmark'

In [ ]:
# Investigate the SMPL params from the PeopleSnapshot dataset
import os
import numpy as np
import pickle

smpl_params_dir = "data/people_snapshot_public/female-3-casual/consensus.pkl"
with open(smpl_params_dir, "rb") as f:
    smpl_params = pickle.load(f, encoding='latin1')
    
print(type(smpl_params))
print(smpl_params.keys())

for k, v in smpl_params.items():
    print(f"{k}: {type(v)}, shape: {v.shape}")

<class 'dict'>
dict_keys(['betas', 'v_personal'])
betas: <class 'numpy.ndarray'>, shape: (10,)
v_personal: <class 'numpy.ndarray'>, shape: (6890, 3)


In [ ]:
import h5py
with h5py.File('data/people_snapshot_public/female-3-casual/reconstructed_poses.hdf5', 'r') as f:
    print(list(f.keys()))  # typically ['ppose', 'trans']
    for k, v in f.items():
        print(f"{k}: {type(v)}, shape: {v.shape}")

['betas', 'pose', 'trans']
betas: <class 'h5py._hl.dataset.Dataset'>, shape: (10,)
pose: <class 'h5py._hl.dataset.Dataset'>, shape: (649, 72)
trans: <class 'h5py._hl.dataset.Dataset'>, shape: (649, 3)


In [1]:
from src.utils.data_loader import mask_loader
mask_dir = "data/people_snapshot_public/female-3-casual/masks.hdf5"
masks = mask_loader(mask_dir)
print(f"masks: {type(masks)}, shape: {masks.shape}")

masks: <class 'numpy.ndarray'>, shape: (648, 1080, 1080)


In [ ]:
import torch
pt_data = torch.load("data/people_snapshot_public/female-3-casual/poses.pt")
for k, v in pt_data.items():
    print(f"{k}: {type(v)}, shape: {v.shape}")

betas: <class 'torch.Tensor'>, shape: torch.Size([10])
thetas: <class 'torch.Tensor'>, shape: torch.Size([649, 72])
transl: <class 'torch.Tensor'>, shape: torch.Size([649, 3])


In [9]:
import numpy as np
# npz_data = np.load("../data/female-3-casual/poses.npz", allow_pickle=True)
npz_data = np.load("data/anim/aist_demo.npz", allow_pickle=True)
for k, v in npz_data.items():
    print(f"{k}: {type(v)}, shape: {v.shape}")

poses: <class 'numpy.ndarray'>, shape: (320, 72)
trans: <class 'numpy.ndarray'>, shape: (320, 3)


In [6]:
# Inverstigate the camera settings
import pickle
cam = pickle.load(open("data/people_snapshot_public/female-1-casual/camera.pkl", "rb"), encoding='latin1')
for k, v in cam.items():
    print(f"{k}: {type(v)}, shape: {v.shape if hasattr(v, 'shape') else 'N/A'}")
    print(v)

camera_k: <class 'numpy.ndarray'>, shape: (5,)
[-0.11669534  0.2515035  -0.00090632 -0.00095365 -0.10548419]
camera_rt: <class 'numpy.ndarray'>, shape: (3,)
[0. 0. 0.]
camera_c: <class 'numpy.ndarray'>, shape: (2,)
[511.78055391 567.12542926]
camera_f: <class 'numpy.ndarray'>, shape: (2,)
[2664.22974522 2664.69277422]
height: <class 'int'>, shape: N/A
1080
width: <class 'int'>, shape: N/A
1080
camera_t: <class 'numpy.ndarray'>, shape: (3,)
[0. 0. 0.]


In [2]:
def load_shape_from_obj(file_path):
    try:
        vertices = []
        faces = []
        with open(file_path) as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith("#"):
                    continue

                parts = line.split()
                if len(parts) == 0:
                    continue

                if parts[0] == "v":
                    vertex = list(map(float, parts[1:]))
                    vertices.append(vertex)
                elif parts[0] == "f":
                    # Handle face indices (OBJ format can have vertex/texture/normal)
                    face = []
                    for part in parts[1:]:
                        # Split by '/' and take the first index (vertex index)
                        vertex_idx = int(part.split("/")[0])
                        face.append(vertex_idx)
                    faces.append(face)

        shape_data = {"vertices": vertices, "faces": faces}

        return shape_data

    except FileNotFoundError:
        print(f"{file_path} not found.")
        return None
    except Exception as e:
        print(f"An error occurred while loading the shape: {e}")
        return None

In [3]:
# Observe SMPL-generated Mesh
obj_path = "../data/female-4-casual/output/embedding.obj"

print("Start investigating mesh generated from SMPL parameters...")
obj_data = load_shape_from_obj(obj_path)
if obj_data:
    print(
        f"Loaded {len(obj_data['vertices'])} vertices and {len(obj_data['faces'])} faces"
    )
    print(f"First vertex: {obj_data['vertices'][0]}")
    print(f"First face: {obj_data['faces'][0]}")
else:
    print("Failed to load OBJ file")

Start investigating mesh generated from SMPL parameters...
Loaded 6890 vertices and 13776 faces
First vertex: [0.171158, -0.709704, 5.285116]
First face: [2, 3, 1]


In [ ]:
# Observe Mesh ==> Gaussian Mapping
import json
json_path = "../data/female-4-casual/output/embedding.json"

print("Investigating mapping from mesh to Gaussian...")
json_data = json.load(open(json_path))
print(f"Loaded JSON data with keys: {list(json_data.keys())}")
for key in json_data:
    print(
        f"{key}: {type(json_data[key]), len(json_data[key]) if hasattr(json_data[key], '__len__') else 'N/A'}"
    )
    
# List out the cano_mesh elements
cano_mesh = json_data.get("cano_mesh", {})
if cano_mesh:
    print(cano_mesh, type(cano_mesh))

Investigating mapping from mesh to Gaussian...
Loaded JSON data with keys: ['cano_mesh', 'sample_fidxs', 'sample_bary', '_xyz', '_rotation']
cano_mesh: (<class 'str'>, 13)
sample_fidxs: (<class 'list'>, 106073)
sample_bary: (<class 'list'>, 106073)
_xyz: (<class 'list'>, 106073)
_rotation: (<class 'list'>, 106073)
embedding.obj <class 'str'>


In [5]:
# Observe Animation file structure(.npz)
# It covers SMPL parameters per frame
import numpy as np 
npz_path = "../data/female-3-casual/poses/anim_nerf_test.npz"

print("Investigating animation .npz file structure...")
npz_data = np.load(npz_path)
print(f"Loaded .npz file with keys: {list(npz_data.keys())}")
for key in npz_data:
    print(
        f"{key}: {type(npz_data[key]), npz_data[key].shape if hasattr(npz_data[key], 'shape') else 'N/A'}"
    )   

Investigating animation .npz file structure...
Loaded .npz file with keys: ['betas', 'global_orient', 'body_pose', 'transl']
betas: (<class 'numpy.ndarray'>, (1, 10))
global_orient: (<class 'numpy.ndarray'>, (51, 3))
body_pose: (<class 'numpy.ndarray'>, (51, 69))
transl: (<class 'numpy.ndarray'>, (51, 3))
